In [288]:
import json
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import re

In [289]:
import colorsys
from matplotlib.colors import to_hex, to_rgb


def scale_lightness(rgb, scale_l):
    rgbhex = False
    if "#" in rgb:
        rgb = to_rgb(rgb)
        rgbhex = True
    # convert rgb to hls
    h, l, s = colorsys.rgb_to_hls(*rgb)
    # manipulate h, l, s values and return as rgb
    c = colorsys.hls_to_rgb(h, min(1, l * scale_l), s=s)
    if rgbhex:
        c = to_hex(c)
    return c

In [290]:
LOCAL = False

if LOCAL:
    local_suffix = "_local"
else:
    local_suffix = ""

In [291]:
%%capture pwd
!pwd

In [292]:
uid = pwd.stdout.split("/")[-1].split("\r")[0]
eco_git_home = (
    "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/"
)
eco_git_path = (
    eco_git_home + "magazine/" + uid + "/data/"
)  #!!! make sure this points to the magazine
vega_embed = requests.get(eco_git_home + "guidelines/html/vega-embed.html").text
colors = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-colors.json").content
)
category_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-category-color.json").content
)
hue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-single-hue-color.json").content
)
mhue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-multi-hue-color.json").content
)
div_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-diverging-color.json").content
)
config = json.loads(
    requests.get(eco_git_home + "guidelines/charts/eco-global-config.json").content
)
height = config["height"]
width = config["width"]
uid, height, width

('issue-2', 300, 500)

# Fig 1

In [293]:
# https://climateactiontracker.org/global/temperatures/
df = pd.read_excel(
    "raw/CAT_2021-05_PublicData_EmissionPathways.xlsx", skiprows=14, nrows=12
).dropna(axis=1, how="all")
df = df.set_index(["Unnamed: 1", "GtCO2e"]).T.reset_index()
df.columns = [
    "year",
    "current_high",
    "current_low",
    "pledges_high",
    "pledges_low",
    "optimistic",
    "c2_high",
    "c2_median",
    "c2_low",
    "c1_high",
    "c1_median",
    "c1_low",
    "historical",
]
df = df.set_index("year").stack().reset_index()
df["scenario"] = df["level_1"].str.split("_").str[0]
df["projection"] = df["level_1"].str.split("_").str[-1]
df = df.set_index(["year", "scenario", "projection"])[0].unstack().reset_index()

/opt/conda/lib/python3.8/site-packages/IPython/core/async_helpers.py:68: PerformanceWarning: indexing past lexsort depth may impact performance.
  coro.send(None)


In [294]:
f = "fig1_climate_scenarios"
f1 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f1.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f1 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

projection,year,scenario,high,historical,low,median,optimistic
0,1990,historical,NaN,35.989277,NaN,NaN,NaN
1,1991,historical,NaN,36.352906,NaN,NaN,NaN
2,1992,historical,NaN,35.529755,NaN,NaN,NaN
3,1993,historical,NaN,35.736477,NaN,NaN,NaN
4,1994,historical,NaN,35.857245,NaN,NaN,NaN


In [295]:
base = alt.Chart(f1).encode(
    x=alt.X(
        "year:Q",
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=5,
            orient="bottom",
            labelAngle=0,
            format=".0f",
            # zindex=1,
            # offset=-43
        ),
    )
)
line1 = (
    base.mark_line(color=colors["eco-gray"])
    .encode(
        y=alt.Y(
            "historical:Q",
            axis=alt.Axis(
                grid=True,
                title="GtCO₂e per year",
                titleAnchor="start",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                titleFontSize=10,
                titleFontWeight="normal",
                ticks=False,
                labelAlign="left",
                labelBaseline="middle",
                labelPadding=-5,
                labelOffset=-10,
                titleX=20,
                titleY=-5,
                titleBaseline="bottom",
                titleAngle=0,
                titleAlign="left",
                tickCount=7,
                format=".0f",
            ),
        )
    )
    .transform_filter('datum.scenario=="historical"')
)
line2 = (
    base.mark_line(color=colors["eco-mid-blue"])
    .encode(y=alt.Y("optimistic:Q"))
    .transform_filter('datum.scenario=="optimistic"')
)
line3 = (
    base.mark_line(color=colors["eco-turquiose"], strokeDash=[5, 5], strokeWidth=1)
    .encode(y=alt.Y("median:Q"))
    .transform_filter('datum.scenario=="c1"')
)
line4 = (
    base.mark_line(color=colors["eco-light-blue"], strokeDash=[5, 5], strokeWidth=1)
    .encode(y=alt.Y("median:Q"))
    .transform_filter('datum.scenario=="c2"')
)
area1 = line3.mark_area(color=colors["eco-turquiose"], opacity=0.6).encode(
    y="low:Q", y2="high:Q"
)
area2 = line4.mark_area(color=colors["eco-light-blue"], opacity=0.6).encode(
    y="low:Q", y2="high:Q"
)
area3 = (
    base.mark_area(color=colors["eco-mid-blue"], opacity=0.6)
    .encode(y="low:Q", y2="high:Q")
    .transform_filter('datum.scenario=="pledges"')
)
area4 = (
    base.mark_area(color=colors["eco-gray"], opacity=0.6)
    .encode(
        y="low:Q",
        y2="high:Q",
    )
    .transform_filter('datum.scenario=="current"')
)
ruler = (
    base.mark_line(color=colors["eco-gray"]).encode(y="y:Q").transform_calculate(y="0")
)
r2021 = (
    base.mark_rule(
        color=scale_lightness(colors["eco-gray"], 1.5),
        strokeDash=[5, 5],
        strokeWidth=0.5,
    )
    .encode(x="x:Q")
    .transform_calculate(x="2021")
)
text2021 = (
    alt.Chart(pd.DataFrame([{"x": 2021, "y": 0, "t": "2021"}]))
    .mark_text(
        color=scale_lightness(colors["eco-gray"], 1.5),
        align="left",
        baseline="bottom",
        dx=5,
        dy=-5,
    )
    .encode(x="x:Q", y="y:Q", text="t:N")
)
text1 = (
    alt.Chart(
        pd.DataFrame(
            [
                {"x": 2100, "y": 43, "t": "Current policies"},
                {"x": 2100, "y": 40, "t": "2.7 - 3.1 °C"},
            ]
        )
    )
    .mark_text(color=colors["eco-gray"], align="left", dx=5)
    .encode(x="x:Q", y="y:Q", text="t:N")
)
text2 = (
    alt.Chart(
        pd.DataFrame(
            [
                {"x": 2100, "y": 21, "t": "Pledges & targets"},
                {"x": 2100, "y": 18, "t": "2.4 °C"},
            ]
        )
    )
    .mark_text(color=colors["eco-mid-blue"], align="left", dx=5)
    .encode(x="x:Q", y="y:Q", text="t:N")
)
text3 = (
    alt.Chart(
        pd.DataFrame(
            [
                {"x": 2100, "y": 14, "t": "Net-zero targets"},
                {"x": 2100, "y": 11, "t": "2.0 °C"},
            ]
        )
    )
    .mark_text(color=colors["eco-mid-blue"], align="left", dx=5)
    .encode(x="x:Q", y="y:Q", text="t:N")
)
text4 = (
    alt.Chart(
        pd.DataFrame(
            [
                {"x": 2100, "y": 2, "t": "<2 °C consistent"},
                {"x": 2100, "y": -1, "t": "1.6 - 1.7 °C"},
            ]
        )
    )
    .mark_text(color=colors["eco-light-blue"], align="left", dx=5)
    .encode(x="x:Q", y="y:Q", text="t:N")
)
text5 = (
    alt.Chart(
        pd.DataFrame(
            [
                {"x": 2100, "y": -5, "t": "<1.5 °C consistent"},
                {"x": 2100, "y": -8, "t": "1.3 °C"},
            ]
        )
    )
    .mark_text(color=colors["eco-turquiose"], align="left", dx=5)
    .encode(x="x:Q", y="y:Q", text="t:N")
)
title = alt.TitleParams(
    "Global greenhouse gas emissions and warming projections until 2100",
    subtitle=[
        "Global CO₂ emissions. Source: Climate Action Tracker, May 2021 briefing"
    ],
    anchor="start",
    align="left",
    dx=5,
    fontSize=12,
    subtitleFontSize=11,
    subtitleFontStyle="italic",
)
layer1 = (
    (
        (
            ruler
            + r2021
            + area1
            + area2
            + area3
            + area4
            + line1
            + line2
            + line3
            + line4
            + text2021
            + text1
            + text2
            + text3
            + text4
            + text5
        ).properties(height=300, width=400)
    )
    .configure_view(stroke=None)
    .properties(title=title)
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "w").write(readme)
layer1

Fontconfig error: Cannot load default config file


alt.LayerChart(...)

In [296]:
base = alt.Chart(f1).encode(
    x=alt.X(
        "year:Q",
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=5,
            titleFontSize=12,
            labelFontSize=12,
            orient="bottom",
            labelAngle=0,
            format=".0f",
            # zindex=1,
            # offset=-43
        ),
    )
)
line1 = (
    base.mark_line(color=colors["eco-gray"])
    .encode(
        y=alt.Y(
            "historical:Q",
            axis=alt.Axis(
                grid=False,
                title="GtCO₂e per year",
                titleAnchor="start",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                titleFontSize=12,
                labelFontSize=12,
                titleFontWeight="normal",
                ticks=False,
                labelAlign="left",
                labelBaseline="middle",
                labelPadding=-5,
                labelOffset=-10,
                titleX=22,
                titleY=-3,
                titleBaseline="bottom",
                titleAngle=0,
                titleAlign="left",
                tickCount=7,
                format=".0f",
            ),
        )
    )
    .transform_filter('datum.scenario=="historical"')
)
line2 = (
    base.mark_line(color=colors["eco-mid-blue"])
    .encode(y=alt.Y("optimistic:Q"))
    .transform_filter('datum.scenario=="optimistic"')
)
line3 = (
    base.mark_line(color=colors["eco-turquiose"], strokeDash=[5, 5], strokeWidth=1)
    .encode(y=alt.Y("median:Q"))
    .transform_filter('datum.scenario=="c1"')
)
line4 = (
    base.mark_line(color=colors["eco-light-blue"], strokeDash=[5, 5], strokeWidth=1)
    .encode(y=alt.Y("median:Q"))
    .transform_filter('datum.scenario=="c2"')
)
area1 = line3.mark_area(color=colors["eco-turquiose"], opacity=0.6).encode(
    y="low:Q", y2="high:Q"
)
area2 = line4.mark_area(color=colors["eco-light-blue"], opacity=0.6).encode(
    y="low:Q", y2="high:Q"
)
area3 = (
    base.mark_area(color=colors["eco-mid-blue"], opacity=0.6)
    .encode(y="low:Q", y2="high:Q")
    .transform_filter('datum.scenario=="pledges"')
)
area4 = (
    base.mark_area(color=colors["eco-gray"], opacity=0.6)
    .encode(
        y="low:Q",
        y2="high:Q",
    )
    .transform_filter('datum.scenario=="current"')
)
ruler = (
    base.mark_line(color=colors["eco-gray"]).encode(y="y:Q").transform_calculate(y="0")
)
r2021 = (
    base.mark_rule(
        color=scale_lightness(colors["eco-gray"], 0.7),
        strokeDash=[5, 5],
        strokeWidth=0.5,
    )
    .encode(x="x:Q")
    .transform_calculate(x="2021")
)
text2021 = (
    alt.Chart(pd.DataFrame([{"x": 2021, "y": 0, "t": "2021"}]))
    .mark_text(
        color=scale_lightness(colors["eco-gray"], 0.7),
        align="left",
        baseline="bottom",
        dx=5,
        dy=-5,
    )
    .encode(x="x:Q", y="y:Q", text="t:N")
)
text1 = (
    alt.Chart(
        pd.DataFrame(
            [
                {"x": 2100, "y": 43, "t": "Current policies"},
                {"x": 2100, "y": 40, "t": "2.7 - 3.1 °C"},
            ]
        )
    )
    .mark_text(color=colors["eco-gray"], align="left", dx=5)
    .encode(x="x:Q", y="y:Q", text="t:N")
)
text2 = (
    alt.Chart(
        pd.DataFrame(
            [
                {"x": 2100, "y": 21, "t": "Pledges & targets"},
                {"x": 2100, "y": 18, "t": "2.4 °C"},
            ]
        )
    )
    .mark_text(color=colors["eco-mid-blue"], align="left", dx=5)
    .encode(x="x:Q", y="y:Q", text="t:N")
)
text3 = (
    alt.Chart(
        pd.DataFrame(
            [
                {"x": 2100, "y": 14, "t": "Net-zero targets"},
                {"x": 2100, "y": 11, "t": "2.0 °C"},
            ]
        )
    )
    .mark_text(color=colors["eco-mid-blue"], align="left", dx=5)
    .encode(x="x:Q", y="y:Q", text="t:N")
)
text4 = (
    alt.Chart(
        pd.DataFrame(
            [
                {"x": 2100, "y": 2, "t": "<2 °C consistent"},
                {"x": 2100, "y": -1, "t": "1.6 - 1.7 °C"},
            ]
        )
    )
    .mark_text(color=colors["eco-light-blue"], align="left", dx=5)
    .encode(x="x:Q", y="y:Q", text="t:N")
)
text5 = (
    alt.Chart(
        pd.DataFrame(
            [
                {"x": 2100, "y": -5, "t": "<1.5 °C consistent"},
                {"x": 2100, "y": -8, "t": "1.3 °C"},
            ]
        )
    )
    .mark_text(color=colors["eco-turquiose"], align="left", dx=5)
    .encode(x="x:Q", y="y:Q", text="t:N")
)
title = alt.TitleParams(
    "Global greenhouse gas emissions and warming projections until 2100",
    subtitle=[
        "Global CO₂ emissions. Source: Climate Action Tracker, May 2021 breifing"
    ],
    anchor="start",
    align="left",
    dx=5,
    dy=-10,
    fontSize=14,
    subtitleFontSize=12,
    color=colors["eco-dot"],
    subtitleColor=colors["eco-dot"],
    subtitleFontStyle="italic",
)
layer1 = (
    (
        (
            ruler
            + r2021
            + area1
            + area2
            + area3
            + area4
            + line1
            + line2
            + line3
            + line4
            + text2021
            + text1
            + text2
            + text3
            + text4
            + text5
        ).properties(height=300, width=400)
    )
    .configure(font="Georgia", background=colors["eco-background"])
    .configure_view(stroke=None)
    .properties(title=title)
)
layer1.save("visualisation/" + f + "_dark.json")
layer1.save("visualisation/" + f + "_dark.png")
readme = re.sub(f, f + "_dark", readme)
open("README.md", "a").write(readme)
layer1

Fontconfig error: Cannot load default config file


alt.LayerChart(...)

# Fig 2

In [297]:
# https://data.worldbank.org/indicator/EN.ATM.CO2E.KD.GD
df = (
    pd.read_csv("raw/API_EN.ATM.CO2E.KD.GD_DS2_en_csv_v2_3016406.csv", skiprows=3)
    .drop(["Indicator Name", "Indicator Code", "Country Code", "Unnamed: 65"], axis=1)
    .set_index("Country Name")
    .stack()
    .reset_index()
)
df.columns = ["country", "year", "value"]

In [298]:
f = "fig2_gdp_emissions_intensity"
f2 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f2.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f2 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,country,year,value
0,Aruba,1986,0.183050
1,Aruba,1987,0.392628
2,Aruba,1988,0.452976
3,Aruba,1989,0.428165
4,Africa Eastern and Southern,1960,0.767570


In [299]:
base = alt.Chart(f2).encode(
    x=alt.X(
        "year:Q",
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=5,
            orient="bottom",
            labelAngle=0,
            format=".0f",
            # zindex=1,
            # offset=-43
        ),
    )
)
lines = (
    base.mark_line()
    .encode(
        y=alt.Y(
            "value:Q",
            axis=alt.Axis(
                grid=True,
                title="kgCO₂e per 2010 US$ of GDP",
                titleAnchor="start",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                titleFontSize=10,
                titleFontWeight="normal",
                ticks=False,
                labelAlign="left",
                labelBaseline="middle",
                labelPadding=-5,
                labelOffset=-10,
                titleX=25,
                titleY=-5,
                titleBaseline="bottom",
                titleAngle=0,
                titleAlign="left",
                tickCount=7,
                format=".1f",
            ),
        ),
        color=alt.Color(
            "country:N",
            legend=None,
            scale=alt.Scale(
                range=[
                    colors["eco-light-blue"],
                    colors["eco-turquiose"],
                    colors["eco-mid-blue"],
                ]
            ),
        ),
    )
    .transform_filter(
        alt.FieldOneOfPredicate(
            field="country", oneOf=["United States", "United Kingdom", "European Union"]
        )
    )
)
line = (
    base.mark_line(color=colors["eco-gray"], strokeWidth=3)
    .encode(y="value:Q")
    .transform_filter(alt.FieldOneOfPredicate(field="country", oneOf=["World"]))
)
labels = (
    lines.mark_text(align="left", dx=5)
    .encode(
        text="country:N",
    )
    .transform_filter("datum.year==2018")
)
label = (
    line.mark_text(
        align="left", dx=5, fontSize=12, fontStyle="bold", color=colors["eco-gray"]
    )
    .encode(text="country:N")
    .transform_filter("datum.year==2018")
)
title = alt.TitleParams(
    "Evolution of the emissions intensity of GDP",
    subtitle=["Emissions intensity of GDP. Source: World Bank Development Indicators"],
    anchor="start",
    align="left",
    dx=5,
    fontSize=12,
    subtitleFontSize=11,
    subtitleFontStyle="italic",
)
layer1 = (
    ((lines + line + labels + label).properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title=title)
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
with alt.data_transformers.disable_max_rows():
    layer1.display()

Fontconfig error: Cannot load default config file


alt.LayerChart(...)

In [300]:
base = alt.Chart(f2).encode(
    x=alt.X(
        "year:Q",
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=5,
            titleFontSize=12,
            labelFontSize=12,
            orient="bottom",
            labelAngle=0,
            format=".0f",
            # zindex=1,
            # offset=-43
        ),
    )
)
lines = (
    base.mark_line()
    .encode(
        y=alt.Y(
            "value:Q",
            axis=alt.Axis(
                grid=False,
                title="kgCO₂e per 2010 US$ of GDP",
                titleAnchor="start",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                titleFontSize=12,
                labelFontSize=12,
                titleFontWeight="normal",
                ticks=False,
                labelAlign="left",
                labelBaseline="middle",
                labelPadding=-5,
                labelOffset=-10,
                titleX=25,
                titleY=-2,
                titleBaseline="bottom",
                titleAngle=0,
                titleAlign="left",
                tickCount=7,
                format=".1f",
            ),
        ),
        color=alt.Color(
            "country:N",
            legend=None,
            scale=alt.Scale(
                range=[
                    colors["eco-light-blue"],
                    colors["eco-turquiose"],
                    colors["eco-mid-blue"],
                ]
            ),
        ),
    )
    .transform_filter(
        alt.FieldOneOfPredicate(
            field="country", oneOf=["United States", "United Kingdom", "European Union"]
        )
    )
)
line = (
    base.mark_line(color=colors["eco-gray"], strokeWidth=3)
    .encode(y="value:Q")
    .transform_filter(alt.FieldOneOfPredicate(field="country", oneOf=["World"]))
)
labels = (
    lines.mark_text(align="left", dx=5)
    .encode(
        text="country:N",
    )
    .transform_filter("datum.year==2018")
)
label = (
    line.mark_text(
        align="left", dx=5, fontSize=12, fontStyle="bold", color=colors["eco-gray"]
    )
    .encode(text="country:N")
    .transform_filter("datum.year==2018")
)
title = alt.TitleParams(
    "Evolution of the emissions intensity of GDP",
    subtitle=["Emissions intensity of GDP. Source: World Bank Development Indicators"],
    anchor="start",
    align="left",
    dx=5,
    dy=-5,
    fontSize=14,
    subtitleFontSize=12,
    color=colors["eco-dot"],
    subtitleColor=colors["eco-dot"],
    subtitleFontStyle="italic",
)
layer1 = (
    ((lines + line + labels + label).properties(height=300, width=400))
    .configure(font="Georgia", background=colors["eco-background"])
    .configure_view(stroke=None)
    .properties(title=title)
)
layer1.save("visualisation/" + f + "_dark.json")
layer1.save("visualisation/" + f + "_dark.png")
readme = re.sub(f, f + "_dark", readme)
open("README.md", "a").write(readme)
with alt.data_transformers.disable_max_rows():
    layer1.display()

Fontconfig error: Cannot load default config file


alt.LayerChart(...)

# Fig 3

In [301]:
# https://www.datagraver.com/case/kyoto-cherry-blossom-full-flower-dates
# https://www1.ncdc.noaa.gov/pub/data/paleo/historical/phenology/japan/LatestVersion/
# https://n-kishou.com/corp/news-contents/sakura/?lang=en
df = pd.read_excel("raw/KyotoFullFlower7.xls", skiprows=25, usecols="A:C").dropna()
df.columns = ["year", "day", "date"]
df["date"] = "0" + df["date"].astype(str).str[0] + "-" + df["date"].astype(str).str[1:3]

In [302]:
f = "fig3_cherry_blossom"
f3 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f3.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f3 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,year,day,date
11,812,92.0,04-01
14,815,105.0,04-15
30,831,96.0,04-06
50,851,108.0,04-18
52,853,104.0,04-14


In [303]:
base = alt.Chart(f3).encode(
    x=alt.X(
        "year:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            title="Year",
            titleAlign="right",
            titleAnchor="end",
            titleY=-15,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            orient="bottom",
            labelAngle=0,
            format=".0f",
        ),
        scale=alt.Scale(domain=[800, 2021], nice=False),
    )
)
points = base.mark_point(
    color=colors["eco-dot"], fill=colors["eco-dot"], size=10, opacity=0.2
).encode(
    y=alt.Y(
        "date:T",
        sort=[],
        axis=alt.Axis(
            grid=False,
            title="First full-flowering day",
            titleX=5,
            titleY=15,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
        scale=alt.Scale(domain=["03-15", "05-15"]),
    )
)
line = (
    points.mark_line(color=colors["eco-dot"])
    .encode(y=alt.Y("y:T", sort=[]))
    .transform_window(y="mean(date)", frame=[-25, 25])
)
line2 = (
    points.mark_line(color=colors["eco-gray"], opacity=0.5)
    .encode(y="y:T")
    .transform_window(y="mean(date)", frame=[-1000, 1000])
)
text1 = (
    alt.Chart(pd.DataFrame([{"x": 810, "y": "04-16", "t": "overall mean"}]))
    .mark_text(color=colors["eco-gray"], align="left", dy=2)
    .encode(x="x:Q", y="y:T", text="t:N")
)
text2 = (
    alt.Chart(pd.DataFrame([{"x": 890, "y": "04-09", "t": "50-year moving window"}]))
    .mark_text(color=colors["eco-dot"], align="left")
    .encode(x="x:Q", y="y:T", text="t:N")
)
text3 = (
    alt.Chart(
        pd.DataFrame(
            [
                {"x": 2021, "y": "04-14", "t": "14th"},
                {"x": 2021, "y": "04-17", "t": "April"},
            ]
        )
    )
    .mark_text(color=colors["eco-gray"], align="left")
    .encode(x="x:Q", y="y:T", text="t:N")
)
text4 = (
    alt.Chart(pd.DataFrame([{"x": 2021, "y": "04-05", "t": "5th"}]))
    .mark_text(color=colors["eco-dot"], align="left")
    .encode(x="x:Q", y="y:T", text="t:N")
)
points2 = points.mark_point(fill=colors["eco-turquiose"]).transform_filter(
    alt.FieldOneOfPredicate(field="year", oneOf=[2021, 1323, 1409])
)
points3 = (
    alt.Chart(pd.DataFrame([{"x": 2021, "y": "03-26"}]))
    .mark_point(fill=colors["eco-turquiose"], opacity=0.8)
    .encode(x="x:Q", y="y:T")
)
text5 = (
    alt.Chart(pd.DataFrame([{"x": 1323, "y": "05-05", "t": "May 5th, 1323"}]))
    .mark_text(
        color=scale_lightness(colors["eco-turquiose"], 0.7), align="center", dy=-8
    )
    .encode(x="x:Q", y="y:T", text="t:N")
)
text6 = (
    alt.Chart(pd.DataFrame([{"x": 1409, "y": "03-27", "t": "March 27th, 1409"}]))
    .mark_text(
        color=scale_lightness(colors["eco-turquiose"], 0.7), align="center", dy=15
    )
    .encode(x="x:Q", y="y:T", text="t:N")
)
text7 = (
    alt.Chart(pd.DataFrame([{"x": 2021, "y": "03-26", "t": "March 26th, 2021"}]))
    .mark_text(
        color=scale_lightness(colors["eco-turquiose"], 0.7), align="right", dx=-7, dy=10
    )
    .encode(x="x:Q", y="y:T", text="t:N")
)
title = alt.TitleParams(
    "Full-flowering day of the cherry blossom in Kyoto, Japan",
    subtitle=[
        "Source: NOAA, based on Aono, Kazui (2008), Aono, Saito (2010) and Aono (2012)"
    ],
    anchor="start",
    align="left",
    dx=5,
    fontSize=12,
    subtitleFontSize=11,
    subtitleFontStyle="italic",
)
layer1 = (
    (
        (
            points
            + line2
            + line
            + text1
            + text2
            + text3
            + text4
            + points2
            + points3
            + text5
            + text6
            + text7
        ).properties(height=300, width=400)
    )
    .configure_view(stroke=None)
    .properties(title=title)
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

WARN An ancestor parsed field "y" as number but a child wants to parse the field as date.
WARN An ancestor parsed field "y" as number but a child wants to parse the field as date.
Fontconfig error: Cannot load default config file


alt.LayerChart(...)

In [304]:
base = alt.Chart(f3).encode(
    x=alt.X(
        "year:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            title="Year",
            titleAlign="right",
            titleAnchor="end",
            titleY=-15,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            titleFontSize=12,
            labelFontSize=12,
            orient="bottom",
            labelAngle=0,
            format=".0f",
        ),
        scale=alt.Scale(domain=[800, 2021], nice=False),
    )
)
points = base.mark_point(
    color=colors["eco-dot"], fill=colors["eco-dot"], size=10, opacity=0.2
).encode(
    y=alt.Y(
        "date:T",
        sort=[],
        axis=alt.Axis(
            grid=False,
            title="First full-flowering day",
            titleX=5,
            titleY=15,
            titleBaseline="bottom",
            titleAngle=0,
            titleFontSize=12,
            labelFontSize=12,
            titleAlign="left",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
        scale=alt.Scale(domain=["03-15", "05-15"]),
    )
)
line = (
    points.mark_line(color=colors["eco-dot"])
    .encode(y=alt.Y("y:T", sort=[]))
    .transform_window(y="mean(date)", frame=[-25, 25])
)
line2 = (
    points.mark_line(color=colors["eco-gray"], opacity=0.5)
    .encode(y="y:T")
    .transform_window(y="mean(date)", frame=[-1000, 1000])
)
text1 = (
    alt.Chart(pd.DataFrame([{"x": 810, "y": "04-16", "t": "overall mean"}]))
    .mark_text(color=colors["eco-gray"], align="left", dy=2)
    .encode(x="x:Q", y="y:T", text="t:N")
)
text2 = (
    alt.Chart(pd.DataFrame([{"x": 890, "y": "04-09", "t": "50-year moving window"}]))
    .mark_text(color=colors["eco-dot"], align="left")
    .encode(x="x:Q", y="y:T", text="t:N")
)
text3 = (
    alt.Chart(
        pd.DataFrame(
            [
                {"x": 2021, "y": "04-14", "t": "14th"},
                {"x": 2021, "y": "04-17", "t": "April"},
            ]
        )
    )
    .mark_text(color=colors["eco-gray"], align="left")
    .encode(x="x:Q", y="y:T", text="t:N")
)
text4 = (
    alt.Chart(pd.DataFrame([{"x": 2021, "y": "04-05", "t": "5th"}]))
    .mark_text(color=colors["eco-dot"], align="left")
    .encode(x="x:Q", y="y:T", text="t:N")
)
points2 = points.mark_point(fill=colors["eco-turquiose"]).transform_filter(
    alt.FieldOneOfPredicate(field="year", oneOf=[2021, 1323, 1409])
)
points3 = (
    alt.Chart(pd.DataFrame([{"x": 2021, "y": "03-26"}]))
    .mark_point(fill=colors["eco-turquiose"], opacity=0.8)
    .encode(x="x:Q", y="y:T")
)
text5 = (
    alt.Chart(pd.DataFrame([{"x": 1323, "y": "05-05", "t": "May 5th, 1323"}]))
    .mark_text(
        color=scale_lightness(colors["eco-turquiose"], 0.7), align="center", dy=-8
    )
    .encode(x="x:Q", y="y:T", text="t:N")
)
text6 = (
    alt.Chart(pd.DataFrame([{"x": 1409, "y": "03-27", "t": "March 27th, 1409"}]))
    .mark_text(
        color=scale_lightness(colors["eco-turquiose"], 0.7), align="center", dy=15
    )
    .encode(x="x:Q", y="y:T", text="t:N")
)
text7 = (
    alt.Chart(pd.DataFrame([{"x": 2021, "y": "03-26", "t": "March 26th, 2021"}]))
    .mark_text(
        color=scale_lightness(colors["eco-turquiose"], 0.7), align="right", dx=-7, dy=10
    )
    .encode(x="x:Q", y="y:T", text="t:N")
)
title = alt.TitleParams(
    "Full-flowering day of the cherry blossom in Kyoto, Japan",
    subtitle=[
        "Source: NOAA, based on Aono, Kazui (2008), Aono, Saito (2010) and Aono (2012)"
    ],
    anchor="start",
    align="left",
    dx=5,
    dy=-5,
    fontSize=14,
    subtitleFontSize=12,
    color=colors["eco-dot"],
    subtitleColor=colors["eco-dot"],
    subtitleFontStyle="italic",
)
layer1 = (
    (
        (
            points
            + line2
            + line
            + text1
            + text2
            + text3
            + text4
            + points2
            + points3
            + text5
            + text6
            + text7
        ).properties(height=300, width=400)
    )
    .configure(font="Georgia", background=colors["eco-background"])
    .configure_view(stroke=None)
    .properties(title=title)
)
layer1.save("visualisation/" + f + "_dark.json")
layer1.save("visualisation/" + f + "_dark.png")
readme = re.sub(f, f + "_dark", readme)
open("README.md", "a").write(readme)
layer1

WARN An ancestor parsed field "y" as number but a child wants to parse the field as date.
WARN An ancestor parsed field "y" as number but a child wants to parse the field as date.
Fontconfig error: Cannot load default config file


alt.LayerChart(...)

# Fig 4

## a

In [305]:
# https://www.eea.europa.eu/data-and-maps/data/external/mass-balance-of-the-greenland
# https://www.epa.gov/climate-indicators/climate-change-indicators-glaciers
# https://wgms.ch/latest-glacier-mass-balance-data/
df = pd.read_csv("raw/glaciers_fig-1.csv", skiprows=6)
df.columns = ["year", "glaciers", "d"]
df = df.drop(["d"], axis=1)
df2 = pd.read_excel("raw/imbie_dataset_greenland_dynamics-2019_12_10.xlsx")
df2 = df2[
    [
        "Year",
        "Cumulative ice sheet mass change (Gt)",
        "Cumulative ice sheet mass change uncertainty (Gt)",
    ]
].dropna()
df2.columns = ["year", "greenland", "e"]
df2 = pd.concat([pd.DataFrame([{"year": 1991, "greenland": 0, "e": 0}]), df2])
df2["greenland1"] = df2["greenland"] - df2["e"] / 2
df2["greenland2"] = df2["greenland"] + df2["e"] / 2
df2 = df2.drop(["e"], axis=1)
df3 = pd.read_excel("raw/imbie_dataset-2018_07_23.xlsx")
df3 = df3[
    [
        "Year",
        "Cumulative ice mass change (Gt)",
        "Cumulative ice mass change uncertainty (Gt)",
    ]
]
df3.columns = ["year", "antarctica", "e"]
df3["antarctica1"] = df3["antarctica"] - df3["e"] / 2
df3["antarctica2"] = df3["antarctica"] + df3["e"] / 2
df3 = df3.drop(["e"], axis=1)
df0 = -df.set_index("year").join(df2.set_index("year")).join(df3.set_index("year"))
df = df0.drop("glaciers", axis=1).reset_index()

In [306]:
f = "fig4a_ice"
f4a = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f4a.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f4a = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,year,greenland,greenland1,greenland2,antarctica,antarctica1,antarctica2
0,1956,NaN,NaN,NaN,NaN,NaN,NaN
1,1957,NaN,NaN,NaN,NaN,NaN,NaN
2,1958,NaN,NaN,NaN,NaN,NaN,NaN
3,1959,NaN,NaN,NaN,NaN,NaN,NaN
4,1960,NaN,NaN,NaN,NaN,NaN,NaN


In [307]:
base = (
    alt.Chart(f4a)
    .encode(
        x=alt.X(
            "year:Q",
            axis=alt.Axis(
                grid=False,
                titleAlign="center",
                titleAnchor="middle",
                title="",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                tickCount=5,
                orient="bottom",
                labelAngle=0,
                format=".0f",
                # zindex=1,
                # offset=-43
            ),
            scale=alt.Scale(domain=[1991, 2020]),
        )
    )
    .transform_filter("datum.year<2018")
    .transform_filter("datum.year>1990")
)
line1 = base.mark_line(color=colors["eco-turquiose"]).encode(
    y=alt.Y(
        "greenland:Q",
        axis=alt.Axis(
            grid=True,
            title="Gt of ice",
            titleAnchor="start",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            titleFontSize=10,
            titleFontWeight="normal",
            ticks=False,
            labelAlign="left",
            labelBaseline="middle",
            labelPadding=-5,
            labelOffset=-10,
            titleX=30,
            titleY=28,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            tickCount=4,
            format=".0f",
        ),
        scale=alt.Scale(domain=[0, 4300]),
    )
)
line2 = base.mark_line(color=colors["eco-light-blue"]).encode(y=alt.Y("antarctica:Q"))
area1 = line1.mark_area(color=colors["eco-turquiose"], opacity=0.5).encode(
    y="greenland1:Q", y2="greenland2:Q"
)
area2 = line2.mark_area(color=colors["eco-light-blue"], opacity=0.5).encode(
    y="antarctica1:Q", y2="antarctica2:Q"
)
ruler = (
    alt.Chart(pd.DataFrame([{"x": 1991, "y": 0}, {"x": 2020, "y": 0}]))
    .mark_line(color=colors["eco-gray"])
    .encode(y="y:Q", x="x:Q")
)
text1 = (
    alt.Chart(pd.DataFrame([{"x": 2017, "y": 3800, "t": "Greenland"}]))
    .mark_text(color=colors["eco-turquiose"], align="left", dx=5)
    .encode(x="x:Q", y="y:Q", text="t:N")
)
text2 = (
    alt.Chart(pd.DataFrame([{"x": 2017, "y": 2600, "t": "Antarctica"}]))
    .mark_text(color=colors["eco-light-blue"], align="left", dx=5)
    .encode(x="x:Q", y="y:Q", text="t:N")
)
title = alt.TitleParams(
    "Cumulative ice mass loss of ice sheets",
    subtitle=["Source: IMBIE/EEA/ESA"],
    anchor="start",
    align="left",
    dx=5,
    fontSize=12,
    subtitleFontSize=11,
    subtitleFontStyle="italic",
)
layer1 = (
    ((area1 + area2 + line1 + line2 + text1 + text2).properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title=title)
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

Fontconfig error: Cannot load default config file


alt.LayerChart(...)

In [308]:
base = (
    alt.Chart(f4a)
    .encode(
        x=alt.X(
            "year:Q",
            axis=alt.Axis(
                grid=False,
                titleAlign="center",
                titleAnchor="middle",
                title="",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                tickCount=5,
                titleFontSize=12,
                labelFontSize=12,
                orient="bottom",
                labelAngle=0,
                format=".0f",
                # zindex=1,
                # offset=-43
            ),
            scale=alt.Scale(domain=[1991, 2020]),
        )
    )
    .transform_filter("datum.year<2018")
    .transform_filter("datum.year>1990")
)
line1 = base.mark_line(color=colors["eco-turquiose"]).encode(
    y=alt.Y(
        "greenland:Q",
        axis=alt.Axis(
            grid=False,
            title="Gt of ice",
            titleAnchor="start",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            titleFontSize=12,
            labelFontSize=12,
            titleFontWeight="normal",
            ticks=False,
            labelAlign="left",
            labelBaseline="middle",
            labelPadding=-5,
            labelOffset=-10,
            titleX=40,
            titleY=31,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            tickCount=4,
            format=".0f",
        ),
        scale=alt.Scale(domain=[0, 4300]),
    )
)
line2 = base.mark_line(color=colors["eco-light-blue"]).encode(y=alt.Y("antarctica:Q"))
area1 = line1.mark_area(color=colors["eco-turquiose"], opacity=0.5).encode(
    y="greenland1:Q", y2="greenland2:Q"
)
area2 = line2.mark_area(color=colors["eco-light-blue"], opacity=0.5).encode(
    y="antarctica1:Q", y2="antarctica2:Q"
)
ruler = (
    alt.Chart(pd.DataFrame([{"x": 1991, "y": 0}, {"x": 2020, "y": 0}]))
    .mark_line(color=colors["eco-gray"])
    .encode(y="y:Q", x="x:Q")
)
text1 = (
    alt.Chart(pd.DataFrame([{"x": 2017, "y": 3800, "t": "Greenland"}]))
    .mark_text(color=colors["eco-turquiose"], align="left", dx=5)
    .encode(x="x:Q", y="y:Q", text="t:N")
)
text2 = (
    alt.Chart(pd.DataFrame([{"x": 2017, "y": 2600, "t": "Antarctica"}]))
    .mark_text(color=colors["eco-light-blue"], align="left", dx=5)
    .encode(x="x:Q", y="y:Q", text="t:N")
)
title = alt.TitleParams(
    "Cumulative ice mass loss of ice sheets",
    subtitle=["Source: IMBIE/EEA/ESA"],
    anchor="start",
    align="left",
    dx=5,
    fontSize=14,
    subtitleFontSize=12,
    color=colors["eco-dot"],
    subtitleColor=colors["eco-dot"],
    subtitleFontStyle="italic",
)
layer1 = (
    ((area1 + area2 + line1 + line2 + text1 + text2).properties(height=300, width=400))
    .configure(font="Georgia", background=colors["eco-background"])
    .configure_view(stroke=None)
    .properties(title=title)
)
layer1.save("visualisation/" + f + "_dark.json")
layer1.save("visualisation/" + f + "_dark.png")
readme = re.sub(f, f + "_dark", readme)
open("README.md", "a").write(readme)
layer1

Fontconfig error: Cannot load default config file


alt.LayerChart(...)

## b

In [309]:
df = df0["glaciers"].reset_index()

In [310]:
f = "fig4b_glaciers"
f4b = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f4b.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f4b = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,year,glaciers
0,1956,-0.000
1,1957,0.120
2,1958,0.976
3,1959,1.414
4,1960,1.968


In [311]:
base = alt.Chart(f4b).encode(
    x=alt.X(
        "year:Q",
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=5,
            orient="bottom",
            labelAngle=0,
            format=".0f",
            # zindex=1,
            # offset=-43
        ),
    )
)
area = base.mark_area(
    interpolate="monotone",
    fillOpacity=0.7,
    line={"color": colors["eco-light-blue"]},
    color=alt.Gradient(
        gradient="linear",
        stops=[
            alt.GradientStop(color="white", offset=0),
            alt.GradientStop(color=colors["eco-light-blue"], offset=0.7),
        ],
        x1=0.8,
        x2=1,
        y1=1,
        y2=0,
    ),
).encode(
    y=alt.Y(
        "glaciers:Q",
        axis=alt.Axis(
            grid=True,
            title="m of water equivalent",
            titleAnchor="start",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            titleFontSize=10,
            titleFontWeight="normal",
            ticks=False,
            labelAlign="left",
            labelBaseline="middle",
            labelPadding=-5,
            labelOffset=-10,
            titleX=20,
            titleY=27,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            tickCount=4,
            format=".0f",
        ),
    )
)
title = alt.TitleParams(
    "Cumulative mass loss of glaciers worldwide",
    subtitle=["Source: EPA, based on WGMS"],
    anchor="start",
    align="left",
    dx=5,
    fontSize=12,
    subtitleFontSize=11,
    subtitleFontStyle="italic",
)
layer1 = (
    ((area).properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title=title)
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

Fontconfig error: Cannot load default config file


alt.Chart(...)

In [312]:
base = alt.Chart(f4b).encode()
area = base.mark_area(
    interpolate="monotone",
    fillOpacity=0.7,
    line={"color": colors["eco-light-blue"]},
    color=alt.Gradient(
        gradient="linear",
        stops=[
            alt.GradientStop(color=colors["eco-background"], offset=0),
            alt.GradientStop(color=colors["eco-light-blue"], offset=0.7),
        ],
        x1=0.8,
        x2=1,
        y1=1,
        y2=0,
    ),
).encode(
    x=alt.X(
        "year:Q",
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            labelFontSize=12,
            title="",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=5,
            orient="bottom",
            labelAngle=0,
            format=".0f",
        ),
    ),
    y=alt.Y(
        "glaciers:Q",
        axis=alt.Axis(
            grid=True,
            gridColor=colors["eco-gray"],
            title="m of water equivalent",
            titleAnchor="start",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            titleFontSize=12,
            labelFontSize=12,
            titleFontWeight="normal",
            ticks=False,
            labelAlign="left",
            labelBaseline="middle",
            labelPadding=-5,
            labelOffset=-10,
            titleX=23,
            titleY=29,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            tickCount=4,
            format=".0f",
        ),
    ),
)
title = alt.TitleParams(
    "Cumulative mass loss of glaciers worldwide",
    subtitle=["Source: EPA, based on WGMS"],
    anchor="start",
    align="left",
    dx=5,
    fontSize=14,
    subtitleFontSize=12,
    color=colors["eco-dot"],
    subtitleColor=colors["eco-dot"],
    subtitleFontStyle="italic",
)
layer1 = (
    (area)
    .properties(height=300, width=400, title=title)
    .configure(font="Georgia", background=colors["eco-background"])
    .configure_view(stroke=None)
)
layer1.save("visualisation/" + f + "_dark.json")
layer1.save("visualisation/" + f + "_dark.png")
readme = re.sub(f, f + "_dark", readme)
open("README.md", "a").write(readme)
layer1

Fontconfig error: Cannot load default config file


alt.Chart(...)

# Fig 6

## a

In [313]:
# climate and productivity
# appendix table A1, available in the online version of the article.
# This should have the underlying results for fig 3a (high risk) and fig 4a (low risk)
df = pd.read_excel("raw/10694apa.xlsx")
df = df.dropna(axis=1, how="all")
for c in df.columns:
    df[c] = (
        df[c]
        .astype(str)
        .str.replace("—", "-")
        .str.replace(" ", "")
        .str.replace(" ", "")
    )
df.columns = ["temp", "l1", "o1", "l2", "o2", "l3", "o3", "o4"]
df = df.set_index("temp")
order = list(df.index[::2])[:-1]
df["columns"] = [i + j * "s" for i in list(df.index[::2]) for j in range(2)][:-1]
df["columns1"] = [j * "s" for i in list(df.index[::2]) for j in range(2)][:-1]
df["columns2"] = [i for i in list(df.index[::2]) for j in range(2)][:-1]
df["columns1"] = df["columns1"].replace("", "a")
df = (
    df.set_index(["columns2", "columns1"]).drop("columns", axis=1).stack().reset_index()
)
df.columns = ["temp", "cat", "measure", "value"]
df = df.set_index(["temp", "measure", "cat"]).unstack().dropna()["value"]
df["s"] = abs(df["s"].astype(float))
df["a"] = df["a"].astype(float)
df["a1"] = df["a"] - 2 * df["s"]
df["a2"] = df["a"] + 2 * df["s"]
df = (
    df.stack()
    .reset_index()
    .set_index(["temp", "measure", "cat"])
    .unstack()[0]
    .reset_index()
)
df = df.set_index("temp").loc[order].reset_index()

In [314]:
# df['t1']=df['temp'].astype(str).str.split('-').str[0]
# df['t2']=df['temp'].astype(str).str.split('-').str[1]
# df['t11']=(df['t1'].replace('<30',0).replace('>100',0).astype(float)-32)/1.8
# df['t22']=(df['t2'].astype(float,errors='ignore')-32)/1.8

In [315]:
f = "fig6a_time-allocation"
f6a = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f6a.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f6a = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

cat,temp,measure,a,a1,a2,s
0,<30,l1,6.423,-13.277,26.123,9.850
1,<30,l2,14.653,-37.431,66.737,26.042
2,<30,l3,1.324,-32.212,34.860,16.768
3,<30,o1,-37.132,-46.240,-28.024,4.554
4,<30,o2,-47.095,-77.729,-16.461,15.317


In [316]:
base = alt.Chart(f6a).encode(
    x=alt.X(
        "temp:O",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="right",
            titleAnchor="end",
            titleFontSize=10,
            titleY=-15,
            titleFontWeight="normal",
            title="maximum temperature (°F)",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=5,
            orient="bottom",
            labelAngle=0,
            # zindex=1,
            # offset=-43
        ),
    )
)
line = (
    base.mark_line(color=colors["eco-mid-blue"], interpolate="basis")
    .encode(
        y=alt.Y(
            "a:Q",
            axis=alt.Axis(
                grid=True,
                title="minutes",
                titleAnchor="start",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                titleFontSize=10,
                titleFontWeight="normal",
                ticks=False,
                labelAlign="left",
                labelBaseline="middle",
                labelPadding=-5,
                labelOffset=-10,
                titleX=20,
                titleY=40,
                titleBaseline="bottom",
                titleAngle=0,
                titleAlign="left",
                tickCount=4,
                format=".0f",
            ),
            scale=alt.Scale(domain=[-120,80])
        )
    )
    .transform_filter('datum.measure=="l2"')
)
area = line.mark_area(
    color=colors["eco-mid-blue"], interpolate="basis", opacity=0.2
).encode(y="a1:Q", y2="a2:Q")
line2 = (
    base.mark_line(color=colors["eco-turquiose"], interpolate="basis")
    .encode(y="a:Q")
    .transform_filter('datum.measure=="o2"')
)
area2 = line2.mark_area(
    color=colors["eco-turquiose"], interpolate="basis", opacity=0.2
).encode(y="a1:Q", y2="a2:Q")
ruler = (
    base.mark_line(color=colors["eco-gray"], strokeDash=[5, 5], opacity=0.5)
    .encode(y="y:Q")
    .transform_calculate(y="0")
)
label1 = line.mark_text(
    color=colors["eco-mid-blue"], text="Labour", align="left", dx=5
).transform_filter('datum.temp==">100"')
label2 = line2.mark_text(
    color=colors["eco-turquiose"], text="Outdoor", align="left", dx=5
).transform_filter('datum.temp==">100"')
label3 = (
    line.mark_text(
        color=colors["eco-gray"], text=">38 °C", align="center", dx=7, fontSize=10
    )
    .encode(y="y:Q")
    .transform_calculate(y="-105")
    .transform_filter('datum.temp==">100"')
)
label4 = (
    line.mark_text(
        color=colors["eco-gray"], text="8-10 °C", align="center", dx=7, fontSize=10
    )
    .encode(y="y:Q")
    .transform_calculate(y="-115")
    .transform_filter('datum.temp=="46-50"')
)
label5 = (
    line.mark_text(
        color=colors["eco-gray"], text="19-21 °C", align="center", dx=7, fontSize=10
    )
    .encode(y="y:Q")
    .transform_calculate(y="-115")
    .transform_filter('datum.temp=="66-70"')
)
label6 = (
    line.mark_text(
        color=colors["eco-gray"], text="< -1 °C", align="center", dx=7, fontSize=10
    )
    .encode(y="y:Q")
    .transform_calculate(y="-115")
    .transform_filter('datum.temp=="<30"')
)
title = alt.TitleParams(
    "Temperature and time allocation",
    subtitle=["High-risk industries. Source: Zivin and Neidell (2021)"],
    anchor="start",
    align="left",
    dx=5,
    fontSize=12,
    subtitleFontSize=11,
    subtitleFontStyle="italic",
)
layer1 = (
    (
        (ruler + area + area2 + line + line2 + label1 + label2 + label3+label4+label5+label6).properties(
            height=300, width=400
        )
    )
    .configure_view(stroke=None)
    .properties(title=title)
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

Fontconfig error: Cannot load default config file


alt.LayerChart(...)

In [317]:
base = alt.Chart(f6a).encode(
    x=alt.X(
        "temp:O",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="right",
            titleAnchor="end",
            titleFontSize=12,
            labelFontSize=12,
            titleY=-17,
            titleX=420,
            titleFontWeight="normal",
            title="maximum temperature (°F)",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=5,
            orient="bottom",
            labelAngle=0,
            # zindex=1,
            # offset=-43
        ),
    )
)
line = (
    base.mark_line(color=colors["eco-light-blue"], interpolate="basis")
    .encode(
        y=alt.Y(
            "a:Q",
            axis=alt.Axis(
                grid=False,
                title="minutes",
                titleAnchor="start",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                titleFontSize=12,
                labelFontSize=12,
                titleFontWeight="normal",
                ticks=False,
                labelAlign="left",
                labelBaseline="middle",
                labelPadding=-5,
                labelOffset=-10,
                titleX=23,
                titleY=42,
                titleBaseline="bottom",
                titleAngle=0,
                titleAlign="left",
                tickCount=4,
                format=".0f",
            ),
            
        scale=alt.Scale(domain=[-120,80])
        )
    )
    .transform_filter('datum.measure=="l2"')
)
area = line.mark_area(
    color=colors["eco-light-blue"], interpolate="basis", opacity=0.2
).encode(y="a1:Q", y2="a2:Q")
line2 = (
    base.mark_line(color=colors["eco-turquiose"], interpolate="basis")
    .encode(y="a:Q")
    .transform_filter('datum.measure=="o2"')
)
area2 = line2.mark_area(
    color=colors["eco-turquiose"], interpolate="basis", opacity=0.2
).encode(y="a1:Q", y2="a2:Q")
ruler = (
    base.mark_line(color=colors["eco-gray"], strokeDash=[5, 5], opacity=0.5)
    .encode(y="y:Q")
    .transform_calculate(y="0")
)
label1 = line.mark_text(
    color=colors["eco-light-blue"], text="Labour", align="left", dx=5
).transform_filter('datum.temp==">100"')
label2 = line2.mark_text(
    color=colors["eco-turquiose"], text="Outdoor", align="left", dx=5
).transform_filter('datum.temp==">100"')
label3 = (
    line.mark_text(
        color=colors["eco-gray"], text=">38 °C", align="center", dx=7, fontSize=12
    )
    .encode(y="y:Q")
    .transform_calculate(y="-102")
    .transform_filter('datum.temp==">100"')
)
label4 = (
    line.mark_text(
        color=colors["eco-gray"], text="8-10 °C", align="center", dx=7, fontSize=12
    )
    .encode(y="y:Q")
    .transform_calculate(y="-112")
    .transform_filter('datum.temp=="46-50"')
)
label5 = (
    line.mark_text(
        color=colors["eco-gray"], text="19-21 °C", align="center", dx=7, fontSize=12
    )
    .encode(y="y:Q")
    .transform_calculate(y="-112")
    .transform_filter('datum.temp=="66-70"')
)
label6 = (
    line.mark_text(
        color=colors["eco-gray"], text="< -1 °C", align="center", dx=7, fontSize=12
    )
    .encode(y="y:Q")
    .transform_calculate(y="-112")
    .transform_filter('datum.temp=="<30"')
)
title = alt.TitleParams(
    "Temperature and time allocation",
    subtitle=["High-risk industries. Source: Zivin and Neidell (2021)"],
    anchor="start",
    align="left",
    dx=5,
    fontSize=14,
    subtitleFontSize=12,
    color=colors["eco-dot"],
    subtitleColor=colors["eco-dot"],
    subtitleFontStyle="italic",
)
layer1 = (
    (
        (ruler + area + area2 + line + line2 + label1 + label2 + label3+label4+label5+label6).properties(
            height=300, width=400
        )
    )
    .configure(font="Georgia", background=colors["eco-background"])
    .configure_view(stroke=None)
    .properties(title=title)
)
layer1.save("visualisation/" + f + "_dark.json")
layer1.save("visualisation/" + f + "_dark.png")
readme=re.sub(f, f+'_dark', readme)
open("README.md", "a").write(readme)
layer1

Fontconfig error: Cannot load default config file


alt.LayerChart(...)

## b

In [318]:
f = "fig6b_time-allocation-b"
f6b = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f6b.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f6b = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

cat,temp,measure,a,a1,a2,s
0,<30,l1,6.423,-13.277,26.123,9.850
1,<30,l2,14.653,-37.431,66.737,26.042
2,<30,l3,1.324,-32.212,34.860,16.768
3,<30,o1,-37.132,-46.240,-28.024,4.554
4,<30,o2,-47.095,-77.729,-16.461,15.317


In [319]:
base = alt.Chart(f6b).encode(
    x=alt.X(
        "temp:O",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="right",
            titleAnchor="end",
            titleFontSize=10,
            titleY=-15,
            titleFontWeight="normal",
            title="maximum temperature (°F)",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=5,
            orient="bottom",
            labelAngle=0,
            # zindex=1,
            # offset=-43
        ),
    )
)
line = (
    base.mark_line(color=colors["eco-mid-blue"], interpolate="basis")
    .encode(
        y=alt.Y(
            "a:Q",
            axis=alt.Axis(
                grid=True,
                title="minutes",
                titleAnchor="start",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                titleFontSize=10,
                titleFontWeight="normal",
                ticks=False,
                labelAlign="left",
                labelBaseline="middle",
                labelPadding=-5,
                labelOffset=-10,
                titleX=20,
                titleY=40,
                titleBaseline="bottom",
                titleAngle=0,
                titleAlign="left",
                tickCount=4,
                format=".0f",
            ),
            
        scale=alt.Scale(domain=[-120,80])
        )
    )
    .transform_filter('datum.measure=="l3"')
)
area = line.mark_area(
    color=colors["eco-mid-blue"], interpolate="basis", opacity=0.2
).encode(y="a1:Q", y2="a2:Q")
line2 = (
    base.mark_line(color=colors["eco-turquiose"], interpolate="basis")
    .encode(y="a:Q")
    .transform_filter('datum.measure=="o3"')
)
area2 = line2.mark_area(
    color=colors["eco-turquiose"], interpolate="basis", opacity=0.2
).encode(y="a1:Q", y2="a2:Q")
ruler = (
    base.mark_line(color=colors["eco-gray"], strokeDash=[5, 5], opacity=0.5)
    .encode(y="y:Q")
    .transform_calculate(y="0")
)
label1 = line.mark_text(
    color=colors["eco-mid-blue"], text="Labour", align="left", dx=5
).transform_filter('datum.temp==">100"')
label2 = line2.mark_text(
    color=colors["eco-turquiose"], text="Outdoor", align="left", dx=5
).transform_filter('datum.temp==">100"')
label3 = (
    line.mark_text(
        color=colors["eco-gray"], text=">38 °C", align="center", dx=7, fontSize=10
    )
    .encode(y="y:Q")
    .transform_calculate(y="-105")
    .transform_filter('datum.temp==">100"')
)
label4 = (
    line.mark_text(
        color=colors["eco-gray"], text="8-10 °C", align="center", dx=7, fontSize=10
    )
    .encode(y="y:Q")
    .transform_calculate(y="-115")
    .transform_filter('datum.temp=="46-50"')
)
label5 = (
    line.mark_text(
        color=colors["eco-gray"], text="19-21 °C", align="center", dx=7, fontSize=10
    )
    .encode(y="y:Q")
    .transform_calculate(y="-115")
    .transform_filter('datum.temp=="66-70"')
)
label6 = (
    line.mark_text(
        color=colors["eco-gray"], text="< -1 °C", align="center", dx=7, fontSize=10
    )
    .encode(y="y:Q")
    .transform_calculate(y="-115")
    .transform_filter('datum.temp=="<30"')
)
title = alt.TitleParams(
    "Temperature and time allocation",
    subtitle=["Low-risk industries. Source: Zivin and Neidell (2021)"],
    anchor="start",
    align="left",
    dx=5,
    fontSize=12,
    subtitleFontSize=11,
    subtitleFontStyle="italic",
)
layer1 = (
    (
        (ruler + area + area2 + line + line2 + label1 + label2 + label3+label4+label5+label6).properties(
            height=300, width=400
        )
    )
    .configure_view(stroke=None)
    .properties(title=title)
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

Fontconfig error: Cannot load default config file


alt.LayerChart(...)

In [320]:
base = alt.Chart(f6).encode(
    x=alt.X(
        "temp:O",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="right",
            titleAnchor="end",
            titleFontSize=12,
            labelFontSize=12,
            titleY=-17,
            titleX=420,
            titleFontWeight="normal",
            title="maximum temperature (°F)",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=5,
            orient="bottom",
            labelAngle=0,
            # zindex=1,
            # offset=-43
        ),
    )
)
line = (
    base.mark_line(color=colors["eco-light-blue"], interpolate="basis")
    .encode(
        y=alt.Y(
            "a:Q",
            axis=alt.Axis(
                grid=False,
                title="minutes",
                titleAnchor="start",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                titleFontSize=12,
                labelFontSize=12,
                titleFontWeight="normal",
                ticks=False,
                labelAlign="left",
                labelBaseline="middle",
                labelPadding=-5,
                labelOffset=-10,
                titleX=23,
                titleY=42,
                titleBaseline="bottom",
                titleAngle=0,
                titleAlign="left",
                tickCount=4,
                format=".0f",
            ),
            
        scale=alt.Scale(domain=[-120,80])
        )
    )
    .transform_filter('datum.measure=="l3"')
)
area = line.mark_area(
    color=colors["eco-light-blue"], interpolate="basis", opacity=0.2
).encode(y="a1:Q", y2="a2:Q")
line2 = (
    base.mark_line(color=colors["eco-turquiose"], interpolate="basis")
    .encode(y="a:Q")
    .transform_filter('datum.measure=="o3"')
)
area2 = line2.mark_area(
    color=colors["eco-turquiose"], interpolate="basis", opacity=0.2
).encode(y="a1:Q", y2="a2:Q")
ruler = (
    base.mark_line(color=colors["eco-gray"], strokeDash=[5, 5], opacity=0.5)
    .encode(y="y:Q")
    .transform_calculate(y="0")
)
label1 = line.mark_text(
    color=colors["eco-light-blue"], text="Labour", align="left", dx=5
).transform_filter('datum.temp==">100"')
label2 = line2.mark_text(
    color=colors["eco-turquiose"], text="Outdoor", align="left", dx=5
).transform_filter('datum.temp==">100"')
label3 = (
    line.mark_text(
        color=colors["eco-gray"], text=">38 °C", align="center", dx=7, fontSize=12
    )
    .encode(y="y:Q")
    .transform_calculate(y="-102")
    .transform_filter('datum.temp==">100"')
)
label4 = (
    line.mark_text(
        color=colors["eco-gray"], text="8-10 °C", align="center", dx=7, fontSize=12
    )
    .encode(y="y:Q")
    .transform_calculate(y="-112")
    .transform_filter('datum.temp=="46-50"')
)
label5 = (
    line.mark_text(
        color=colors["eco-gray"], text="19-21 °C", align="center", dx=7, fontSize=12
    )
    .encode(y="y:Q")
    .transform_calculate(y="-112")
    .transform_filter('datum.temp=="66-70"')
)
label6 = (
    line.mark_text(
        color=colors["eco-gray"], text="< -1 °C", align="center", dx=7, fontSize=12
    )
    .encode(y="y:Q")
    .transform_calculate(y="-112")
    .transform_filter('datum.temp=="<30"')
)
title = alt.TitleParams(
    "Temperature and time allocation",
    subtitle=["Low-risk industries. Source: Zivin and Neidell (2021)"],
    anchor="start",
    align="left",
    dx=5,
    fontSize=14,
    subtitleFontSize=12,
    color=colors["eco-dot"],
    subtitleColor=colors["eco-dot"],
    subtitleFontStyle="italic",
)
layer1 = (
    (
        (ruler + area + area2 + line + line2 + label1 + label2 + label3+label4+label5+label6).properties(
            height=300, width=400
        )
    )
    .configure(font="Georgia", background=colors["eco-background"])
    .configure_view(stroke=None)
    .properties(title=title)
)
layer1.save("visualisation/" + f + "_dark.json")
layer1.save("visualisation/" + f + "_dark.png")
readme=re.sub(f, f+'_dark', readme)
open("README.md", "a").write(readme)
layer1

Fontconfig error: Cannot load default config file


alt.LayerChart(...)

# Fig 7

In [321]:
# firms and competitiveness